In [41]:
import redis
import pymysql
import re

In [23]:
def connect_to_db(DB_NAME):
    host = '192.168.1.52'
    port = 3306
    user = 'recipe'
    passwd = 'recipe'
    db = DB_NAME
    charset = 'utf8mb4'
    conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db, charset=charset)
    print('Successfully connected to DB : {} !'.format(DB_NAME))
    return conn

In [75]:
db = 'recipe'
conn = connect_to_db(db)
cursor = conn.cursor()

Successfully connected to DB : recipe !


In [ ]:
sql = """
SELECT * FROM recipe;
"""
cursor.execute(sql)

In [ ]:
data = cursor.fetchall()

In [ ]:
len(data)

In [ ]:
data[:10]

In [ ]:
sql='''SELECT `保存期限` FROM recipe.ingredient where `食材ID` = 50;'''
cursor.execute(sql)
expire = cursor.fetchall()

In [ ]:
word = expire[0][0]

In [ ]:
pattern = r"(.*)([\d]+)"
expire_date = re.search(pattern,word).group()

In [ ]:
int(expire_date)

In [ ]:
userid = 'user1'
ingredient_id = "50"
ing_weight= "250"
unit = "gram"
storage_date = '2020-10-31'
expire_date

In [ ]:
# 新增食材寫入買西摳
sql='''
INSERT INTO `recipe`.`refrigerator_record` 
(`使用者ID`, `食材ID`, `食材重量`, `食材單位`, `食材存放日`,`食材到期日`) 
VALUES ("{}","{}","{}","{}","{}", "{}" + interval "{}" day ) ;

'''.format(userid,ingredient_id,ing_weight,unit,storage_date,storage_date,expire_date)


In [ ]:
cursor.execute(sql)
conn.commit()

In [ ]:
sql_insert = """
INSERT INTO recipe
VALUES (%s,%s,%s);
"""
print('新增資料筆數:', cursor.executemany(sql_insert, recipe_list))

In [19]:
# 查詢食材名稱對應的id及保存期限
a = "飯"
ing_sql = '''
select * from recipe.ingredient where `食材名稱` = '{}';
'''.format(a)
cursor.execute(ing_sql)
ing_info = cursor.fetchall()
ing_id, ing_name, expire_date = ing_info[0]
pattern = r"([\D]*)([\d]+)"
ex_day = re.match(pattern,expire_date).group(2)

In [20]:
print(ing_info)
print(ing_id)
print(ing_name)
print(expire_date)
print(ex_day)

((1, '飯', '3天'),)
1
飯
3天
3


In [55]:
sql = '''
SELECT `使用者ID`, `Line_ID` from recipe.user_profile where `Line_ID` = 'U429ec102b46a5332b32c4f1a8b3b04db';'''
cursor.execute(sql)
user_info = cursor.fetchall()

In [62]:
user_info

(('Ryan', 'U429ec102b46a5332b32c4f1a8b3b04db'),)

In [45]:
sql='''
select 密碼 from user_profile;'''
cursor.execute(sql)
result = cursor.fetchall()

In [46]:
result

(('',), ('',), ('',), ('',), ('',), ('',), ('',), ('',), ('',), ('',))

In [39]:
{x[1]:x[0] for x in result}

{'甜味': 1,
 '鹹味': 2,
 '苦味': 3,
 '辣味': 4,
 '酸味': 5,
 '日式': 6,
 '台式': 7,
 '中式': 8,
 '韓式': 9,
 '東南亞式': 10,
 '美式': 11,
 '歐式': 12,
 '英式': 13,
 '價格便宜': 14,
 '簡單快速出好菜': 15,
 '依據其他人的推薦,評價': 16,
 '根據自我的口味喜好': 17}

In [71]:
# 取得現存食材資訊
sql = '''
SELECT re.使用者ID, ing.食材名稱, re.食材重量, re.食材單位, re.食材存放日, re.食材到期日 
FROM refrigerator_record re JOIN ingredient ing 
ON re.食材ID = ing.食材ID 
WHERE (re.使用者ID = '{}' AND re.食材取用日 is null);
'''.format("user2")
cursor.execute(sql)
result = cursor.fetchall()

In [72]:
result

(('user2',
  '肉燥',
  250,
  'gram',
  datetime.date(2020, 10, 5),
  datetime.date(2020, 10, 15)),)

In [93]:
# 抓特定使用者冰箱資料
sql = '''
SELECT us.Line_ID, re.食材ID, ing.食材名稱, re.食材重量, re.食材單位, re.食材存放日, re.食材到期日 
FROM refrigerator_record re JOIN ingredient ing JOIN user_profile us
ON re.食材ID = ing.食材ID AND re.使用者ID = us.使用者ID
WHERE (re.使用者ID = '{}' AND re.食材取用日 is null);
'''.format("user8")
cursor.execute(sql)
result2 = cursor.fetchall()

In [94]:
result2

()

In [92]:
str(result2[0][3]) + "," + result2[0][4] + "," + str(result2[0][5])

'250,gram,2020-10-05'

In [83]:
# 抓全部冰箱資料
sql = '''
SELECT re.使用者ID, us.Line_ID, re.食材ID, ing.食材名稱, re.食材重量, re.食材單位, re.食材存放日, re.食材到期日 
FROM refrigerator_record re JOIN ingredient ing JOIN user_profile us
ON re.食材ID = ing.食材ID AND re.使用者ID = us.使用者ID
WHERE (re.食材取用日 is null);
'''
cursor.execute(sql)
result3 = cursor.fetchall()

In [84]:
result3

(('user2',
  '777',
  50,
  '肉燥',
  250,
  'gram',
  datetime.date(2020, 10, 5),
  datetime.date(2020, 10, 15)),
 ('user1',
  '111',
  50,
  '肉燥',
  250,
  'gram',
  datetime.date(2020, 10, 5),
  datetime.date(2020, 10, 15)))

In [95]:
cursor.close()
conn.close()